In [1]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import train_cluster_cifar_dfca_tf as DFCA
import train_cluster_cifar_tf as IFCA

import numpy as np

from util import *


Instructions for updating:
non-resource variables are not supported in the long term


Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [2]:
rc = ipp.Cluster(n=2).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 2/2 [00:06<00:00,  3.18s/engine]


In [ ]:
# theoretically irrelevant here because we initialize the model seed randomly (see find_good_initializer())
seeds = [11,13,15]
p_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

dfca_gi = []
ifca = []

In [ ]:
%%px --targets all

import sys
import os
sys.path.append(os.getcwd())

# theoretically irrelevant here because we initialize the model seed randomly (see find_good_initializer())
seeds = [11,13,15]
p_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

dfca_gi = []
ifca = []

Running the dfca-gi experiment

In [ ]:
%%px --target 0 --noblock
import train_cluster_cifar_dfca_tf as DFCA
import json
import time
import torch
import numpy as np


with open("config_pn.json", "r") as read_file:
    config = json.load(read_file)

for pr in p_rates:
    res_final = []
    config['participation_rate'] = pr

    for seed in seeds:
        print(f"---------------------------------DFCA-GI, Participation Rate: {pr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainCIFARCluster(config)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [ ]:
%%px --target 1 --noblock
import train_cluster_cifar_tf as IFCA
import json
import time
import torch
import numpy as np

with open("config_pn.json", "r") as read_file:
    config = json.load(read_file)

for pr in p_rates:
    res_final = []
    config['participation_rate'] = pr

    for seed in seeds:
        print(f"---------------------------------IFCA, Participation Rate: {pr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = IFCA.TrainCIFARCluster(config)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res], [r['train']['loss'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    ifca.append(res_mean)



<AsyncResult(%px): pending>

## Results

DFCA-GI Experiment Outputs

In [7]:
rc.wait_interactive()
_.display_outputs()

dfca_gi = rc[0].pull('dfca_gi', block=True)
ifca = rc[1].pull('ifca', block=True)

unknown: 100%|██████████| 1/1 [00:44<00:00, 44.19s/tasks]


IFCA Experiment Outputs

In [8]:
__.display_outputs()


Instructions for updating:
non-resource variables are not supported in the long term
hi1
hi2
---------------------------------DFCA-GI, Participation Rate: 0.1, seed: 11---------------------------------
config: {'m': 200, 'm_test': 40, 'p': 2, 'n': 500, 'participation_rate': 0.1, 'num_epochs': 1, 'batch_size': 50, 'tau': 5, 'lr': 0.25, 'data_seed': 11, 'train_seed': 11, 'project_dir': 'output'}
Using device: cuda

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



finding good initializer from train data
Epoch -1 tr: l 4.676 a 0.102  clct[np.int64(5), np.int64(15)] ans[np.int64(10), np.int64(10)] cl_acc 0.550 3.142sec
Epoch -1 tr: l 4.673 a 0.108  clct[np.int64(11), np.int64(9)] ans[np.int64(12), np.int64(8)] cl_acc 0.750 2.851sec
found good initializer
Epoch -1 tr: l 4.674 a 0.110  clct[np.int64(5), np.int64(15)] ans[np.int64(7), np.int64(13)] cl_acc 0.800 2.771sec
Epoch -1 tst: l 4.674 a 0.110  clct[np.int64(16), n

## Plots

In [ ]:

r_ifca = {}
r_dfca_gi = {}

r_ifca['test_loss'] = []
r_ifca['test_acc'] = []
r_ifca['cl_acc'] = []
r_ifca['train_loss'] = []
for i in range(len(p_rates)):
    r_ifca['test_loss'].append(ifca[i][0])
    r_ifca['test_acc'].append(ifca[i][1])
    r_ifca['cl_acc'].append(ifca[i][2])
    r_ifca['train_loss'].append(ifca[i][3])

r_dfca_gi['test_loss'] = []
r_dfca_gi['test_acc'] = []
r_dfca_gi['cl_acc'] = []
r_dfca_gi['train_loss'] = []
for i in range(len(p_rates)):
    r_dfca_gi['test_loss'].append(dfca_gi[i][0])
    r_dfca_gi['test_acc'].append(dfca_gi[i][1])
    r_dfca_gi['cl_acc'].append(dfca_gi[i][2])
    r_dfca_gi['train_loss'].append(dfca_gi[i][3])

print("DFCA-GI Test Acc: ", r_dfca_gi['test_acc'])
print("IFCA Test Acc: ", r_ifca['test_acc'])

os.makedirs('graphs', exist_ok=True)

for l in range(len(p_rates)):

    # plt.figure(figsize=(10,5))
    # plt.plot(r_ifca['train_loss'][l], label='ifca', color='blue')
    # plt.plot(r_dfca_gi['train_loss'][l], label='dfca-gi', color='green')
    # plt.xlabel('epoch')
    # plt.ylabel('loss')
    # plt.title(f'Train Loss per Epoch (lr={p_rates[l]})')
    # plt.legend()
    # plt.grid(True)
    # plt.savefig(os.path.join('graphs', '3_train_loss.png'))

    # plt.figure(figsize=(10,5))
    # plt.plot(r_ifca['test_loss'][l], label='ifca', color='blue')
    # plt.plot(r_dfca_gi['test_loss'][l], label='dfca-gi', color='green')
    # plt.xlabel('epoch')
    # plt.ylabel('loss')
    # plt.title(f'Test Loss per Epoch (lr={p_rates[l]})')
    # plt.legend()
    # plt.grid(True)
    # plt.savefig(os.path.join('graphs', '3_test_loss.png'))

    # plt.figure(figsize=(10,5))
    # plt.plot(r_ifca['test_acc'][l], label='ifca', color='blue')
    # plt.plot(r_dfca_gi['test_acc'][l], label='dfca-gi', color='green')
    # plt.xlabel('epoch')
    # plt.ylabel('test accuracy')
    # plt.title(f'Test Accuracy per Epoch (lr={p_rates[l]})')
    # plt.legend()
    # plt.grid(True)
    # plt.savefig(os.path.join('graphs', '3_test_acc.png'))

    plt.figure(figsize=(10,5))
    plt.plot(r_ifca['cl_acc'][l], label='ifca', color='blue')
    plt.plot(r_dfca_gi['cl_acc'][l], label='dfca-gi', color='green')
    plt.xlabel('epoch')
    plt.ylabel('cluster acc')
    plt.title(f'Cluster Accuracy per Epoch (participation_rate={p_rates[l]})')
    plt.legend()
    plt.grid(True)



IFCA Test Acc:  []
DFCA-GI Test Acc:  []


IndexError: list index out of range